In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error
#from  typing import
import warnings
warnings.filterwarnings('ignore')
from scipy import sparse
#!pip install xgboost optuna scikit-learn numpy pandas

In [3]:
df=pd.read_csv("data/nonull_smalldata.csv")
df.columns
df.dtypes
numeric_columns=["bedroomCount","toilet_and_bath","habitableSurface","facedeCount","hasTerrace","totalParkingCount"]
categorical_columns=["type","subtype","province","locality","postCode","buildingCondition","epcScore"]

encoder = OneHotEncoder(sparse_output=False, drop="first")
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
df = pd.concat([df, one_hot_df], axis=1)
df = df.drop(categorical_columns, axis=1)
#df.to_csv("data/imputed_encoded_data.csv")

X=df.drop(columns="price")
y=df["price"]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=1234)
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled= scaler.transform(X_test)
#X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
#X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
X_test = pd.DataFrame(X_test, columns=X_test.columns)

xgb_regressor = XGBRegressor(
    objective='reg:squarederror', # For regression tasks, specify the objective
    n_estimators=180,             # Number of boosting rounds
    learning_rate=0.1,            # Step size shrinkage
    max_depth=4,                  # Maximum depth of a tree
    subsample=0.6,                # Subsample ratio of the training instance
    colsample_bytree=0.6,         # Subsample ratio of columns
    random_state=42,              # For reproducibility
    n_jobs=-1                     # Use all available CPU cores
)

# Fit the model to the training data
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test data
train_preds = xgb_regressor.predict(X_train)
test_preds = xgb_regressor.predict(X_test)

# Evaluate using stored predictions
print("XGBoost MSE (test):", mean_squared_error(y_test, test_preds))
print("R2 Score (test):", r2_score(y_test, test_preds))
print("R2 Score (train):", r2_score(y_train, train_preds))
print("MAE (test):", mean_absolute_error(y_test, test_preds))
print("MAE (train):", mean_absolute_error(y_train, train_preds))
print("MAPE (test):", mean_absolute_percentage_error(y_test, test_preds))
print("MAPE (train):", mean_absolute_percentage_error(y_train, train_preds))

XGBoost MSE (test): 52528712657.64994
R2 Score (test): 0.7990916265311266
R2 Score (train): 0.8376644338462151
MAE (test): 112546.4594272763
MAE (train): 107312.24248498713
MAPE (test): 0.25560821305746084
MAPE (train): 0.2515677916508978


In [ ]:
param_grid = {
    'n_estimators': [100],
    'max_depth': [5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1, 10]
}

xgb = XGBRegressor(random_state=42, n_jobs=2)

# Grid search
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=5,
    verbose=2
)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_
